# Artículo para Science of the Total Environment

* Ajustar PCR
* Ajustar PLS
* Métricas de cross-validation

**Idea general**

Podemos hacer índices en diferentes momentos usando por lo menos dos métodos diferentes.

* PLS
* PCR

Vamos a ajustar modelos con ambos métodos y compararlos a partir de su capacidad predictiva usando validaión cruzada. Vamos a probar cada método en dos escenarios diferentes, primero usando una selección muy pequeña de variables, inspirada directamente en Archarya y luego con una selección más amplia (pero de todas formas más chica que la que usamos originalmente). Con nestos dos escenarios tratamos de ver qué metodos es mejor para manejar nueva información que es seguramente muy colineal con la anterior.

In [ ]:
from ipywidgets import IntProgress
from IPython.display import display
import time

from covid19_vulnerabilidad_mex.datos import *
from covid19_vulnerabilidad_mex.vulnerabilidad import *

import pandas as pd
import geopandas as gpd

import glob
import os
import matplotlib.pyplot as plt
import numpy as np

from datetime import timedelta
import datetime
from datetime import timedelta, date, datetime
import pickle

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.cross_decomposition import PLSRegression

import seaborn as sns
import imageio
from pygifsicle import optimize

plt.style.use('ggplot')

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
## Leemos datos y fijamos los parámetros generales
antier = date.today() - timedelta(days=2)
fecha_analisis = antier - timedelta(weeks=2)
fecha_analisis = pd.to_datetime(fecha_analisis)
inicio = pd.to_datetime('20-05-01', yearfirst=True)
fin = antier - timedelta(days=7)
metodo = 'PLS'
ventana_dias = 28
serie_municipal_base = pd.read_pickle("datos/nuevo_articulo/serie_municipal_base.pkl")
serie_municipal_extended = pd.read_pickle("datos/nuevo_articulo/serie_municipal_extended.pkl")

## Regresión de Componentes Principales

In [ ]:
serie_municipal_base.columns

Index(['FECHA_INGRESO', 'ENTIDAD_RES', 'CLAVE_ENTIDAD_RES', 'MUNICIPIO_RES',
       'CLAVE_MUNICIPIO_RES', 'RESULTADO', 'conteo', 'defunciones', 'geometry',
       '60 and older', 'Disabled', 'Dwellers per room',
       'Índice de rezago social', 'Poverty', 'Educational lag',
       'Health access lag', 'Household characteristics lag',
       'Basic services lag', 'Beds in public hospitals per (state) habitant',
       'Beds in private hospitals per (state) habitant',
       'Number of public hospitals per (state) habitant',
       'People with hypertension', 'People with diabetes',
       'People with obesity', 'Population density', 'Urbanization',
       'Population', 'covid_confirmados_100k', 'covid_defun_100k',
       'tasa_covid_letal'],
      dtype='object')